# Render Contours

author = Caroline Magg <br>
date = 01 April 2020 <br>

___________________________________
history: <br>
2020-04-01 <br>

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import pydicom
import vtk

In [ ]:
%matplotlib inline

## Add path dependencies

In [ ]:
# add KidsBrainProject main folder to paths
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../utils/'))

In [ ]:
from utils_explore import read_structure,read_contour,read_contour_names,read_contour_row

In [ ]:
# add path to data here
path_data = "../../Data/" 

In [ ]:
idx = 1
folder_name = os.path.join(path_data, str(idx))
folder = os.listdir(folder_name)
folder

## Create Contour Information

In [ ]:
path_contours = os.path.join(path_data, str(idx), 'RS.Jacks1.dcm')
df_contours = read_contour_names(path_contours)
#df_contours = df_contours.sort_values(by=['RoiNumber'])
len(df_contours)

In [ ]:
df_contours

In [ ]:
df_test = df_contours.iloc[0:2]
df_test

In [ ]:
contour = read_contour_row(path_contours, df_test)

In [ ]:
len(contour)

In [ ]:
np.unique(contour[:,2])

In [ ]:
test = []
for i in range(len(contour)):
    if contour[i][2] == 203.5:
        test.append(contour[i])
test = np.array(test)
mask = np.zeros((500,200))
for i in range(len(test)):
    mask[int(test[i][0]),int(test[i][1])] = 1
plt.imshow(mask)

In [ ]:
test

In [ ]:
actors = []
for i in range(len(df_test)): 
    # create contour coordinates
    df_t = df_test[i:i+1]
    contour = read_contour_row(path_contours, df_t)
    # save points as vtkPolyData
    points = vtk.vtkPoints()
    for idx, point in enumerate(contour):
        points.InsertPoint(idx, point[0], point[1], point[2])
    pointCloud = vtk.vtkPolyData()
    pointCloud.SetPoints(points)
    # Create triangulation
    delaunay = vtk.vtkDelaunay2D() # Vtk.vtkDelaunay3D() is not working
    delaunay.SetInputData(pointCloud)
    delaunay.SetTolerance(0.05)
    delaunay.SetAlpha(0.2)
    #delaunay.BoundingTriangulationOff()
    delaunay.Update()
    # extracts outer (polygonal) surface
    surfaceFilter = vtk.vtkDataSetSurfaceFilter()
    surfaceFilter.SetInputConnection(delaunay.GetOutputPort())
    surfaceFilter.Update()
    # Mapper
    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputData(surfaceFilter.GetOutput())
    mapper.Update()

    # Actor
    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(i, 0, 0)
    actors.append(actor)

# Renderer
renderer = vtk.vtkRenderer()
for i in range(len(actors)):
    renderer.AddActor(actors[i])
renderer.SetBackground(.2, .3, .3)  #colour
renderer.ResetCamera()

# Render Window
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)

# Interactor
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)

# Begin Interaction
renderWindow.Render()
renderWindowInteractor.Start()

In [ ]:
#contour = read_contour_row(path_contours, df_test)

In [ ]:
len(contour)

In [ ]:
## Single contour

# save points as vtkPolyData
points = vtk.vtkPoints()
for idx, point in enumerate(contour):
    points.InsertPoint(idx, point[0], point[1], point[2])
pointCloud = vtk.vtkPolyData()
pointCloud.SetPoints(points)

# Create triangulation
delaunay = vtk.vtkDelaunay2D() # Vtk.vtkDelaunay3D() is not working
delaunay.SetInputData(pointCloud)
delaunay.SetTolerance(0.05)
delaunay.SetAlpha(0.2)
#delaunay.BoundingTriangulationOff()
delaunay.Update()

# extracts outer (polygonal) surface
surfaceFilter = vtk.vtkDataSetSurfaceFilter()
surfaceFilter.SetInputConnection(delaunay.GetOutputPort())
surfaceFilter.Update()

# Mapper
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputData(surfaceFilter.GetOutput())
mapper.Update()

# Actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor(1, 0, 0)

# Renderer
renderer = vtk.vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(.2, .3, .3)  #colour
renderer.ResetCamera()

# Render Window
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)

# Interactor
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)

# Begin Interaction
renderWindow.Render()
renderWindowInteractor.Start()

In [ ]:
# This example shows how to use Delaunay3D with alpha shapes.

import vtk

# The points to be triangulated are generated randomly in the unit
# cube located at the origin. The points are then associated with a
# vtkPolyData.
math = vtk.vtkMath()
points = vtk.vtkPoints()
for i in range(0, 25):
    points.InsertPoint(i, math.Random(0, 1), math.Random(0, 1),
                       math.Random(0, 1))

profile = vtk.vtkPolyData()
profile.SetPoints(points)

# Delaunay3D is used to triangulate the points. The Tolerance is the
# distance that nearly coincident points are merged
# together. (Delaunay does better if points are well spaced.) The
# alpha value is the radius of circumcircles, circumspheres. Any mesh
# entity whose circumcircle is smaller than this value is output.
delny = vtk.vtkDelaunay3D()
delny.SetInputData(profile)
delny.SetTolerance(0.01)
delny.SetAlpha(0.2)
delny.BoundingTriangulationOff()

# Shrink the result to help see it better.
shrink = vtk.vtkShrinkFilter()
shrink.SetInputConnection(delny.GetOutputPort())
shrink.SetShrinkFactor(0.9)

map = vtk.vtkDataSetMapper()
map.SetInputConnection(shrink.GetOutputPort())

triangulation = vtk.vtkActor()
triangulation.SetMapper(map)
triangulation.GetProperty().SetColor(1, 0, 0)

# Create graphics stuff
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

# Add the actors to the renderer, set the background and size
ren.AddActor(triangulation)
ren.SetBackground(1, 1, 1)
renWin.SetSize(250, 250)
renWin.Render()

cam1 = ren.GetActiveCamera()
cam1.Zoom(1.5)

iren.Initialize()
renWin.Render()
iren.Start()